# Data Privacy Final Project
### Sam Clark & Josh Childs

For this project, we've decided to compare the accuracy of several normal Convolutional Neural Networks to their counter parts that will use differential privacy. We will be using the MNIST dataset with the tensflow library.  

In [472]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
from ipywidgets import IntProgress
from sklearn.metrics import classification_report
import numpy as np
tf.enable_v2_behavior()

## Load MNIST Data

In [473]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

## Preprocess Data

In [474]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## Build Model

In [475]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(28, kernel_size=(3,3), input_shape=input_shape),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation=tf.nn.relu),  
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(10,activation=tf.nn.softmax)
])
model.summary()

Model: "sequential_54"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_54 (Conv2D)           (None, 26, 26, 28)        280       
_________________________________________________________________
max_pooling2d_54 (MaxPooling (None, 13, 13, 28)        0         
_________________________________________________________________
flatten_54 (Flatten)         (None, 4732)              0         
_________________________________________________________________
dense_108 (Dense)            (None, 128)               605824    
_________________________________________________________________
dropout_54 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_109 (Dense)            (None, 10)                1290      
Total params: 607,394
Trainable params: 607,394
Non-trainable params: 0
_______________________________________________

## Train & Save

In [476]:
# callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

# model.compile(optimizer='sgd',
#               loss='sparse_categorical_crossentropy', 
#               metrics=['accuracy'])

# model.fit(x=x_train,y=y_train, epochs=1, callbacks = [callback])
# model.save("models/original")

## Evaluate Models

In [477]:
# model.evaluate(x_test, y_test)

## Differential Privacy Optimizer Implementation

In [478]:
def tf_l2_clip(v, b):
    norm = tf.norm(v, ord=2)
    return tf.cond(norm > b, lambda: b * (v / norm), lambda: v)


def tf_gaussian_mech(v, sensitivity, epsilon, delta):
    return v + tf.random.normal(v.shape, mean=0.0, stddev=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def tf_gaussian_mech_RDP(v, sensitivity, alpha, epsilon):
    sigma = np.sqrt((sensitivity**2 * alpha) / (2 * epsilon))
    return v + tf.random.normal(v.shape, mean=0.0, stddev=sigma)

def tf_gaussian_mech_zCDP(v, sensitivity, rho):
    sigma = np.sqrt((sensitivity**2) / (2 * rho))
    return v + tf.random.normal(v.shape, mean=0.0, stddev=sigma)

In [479]:
class DPOptimizer(tf.keras.optimizers.Optimizer):
    def __init__(self, epochs, b=3.0, learning_rate=0.01, name="DPOptimizer", **kwargs):
        super().__init__(name, **kwargs)
        self._set_hyper("learning_rate", learning_rate)
        self.epochs = epochs
        self.b = b
    
    def _create_slots(self, var_list):
        pass

    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            "learning_rate": self._serialize_hyperparameter("learning_rate"),
        }

    
class EpsilonDeltaDPGradientDescent(DPOptimizer):
    def __init__(self, epochs, epsilon, delta, b=3.0, learning_rate=0.01, name="EpsilonDeltaDPGradientDescent", **kwargs):
        DPOptimizer.__init__(self, epochs, b=b, learning_rate=learning_rate, name=name, **kwargs)        
        self.epsilon = epsilon
        self.delta = delta

    @tf.function
    def _resource_apply_dense(self, grad, var):
        var_dtype = var.dtype.base_dtype
        lr_t = self._decayed_lr(var_dtype)
        
        epsilon_i = self.epsilon / self.epochs
        print(self.epochs)
        delta_i = self.delta / self.epochs
        print(delta_i)
        
        #clipped_grad = tf.math.reduce_mean(tf_l2_clip(grad, self.b), axis=0)
        #clipped_grad = tf.numpy_function(lambda x: np.mean(x, axis=0), [tf_l2_clip(grad, self.b)], tf.float32)
        #clipped_grad = tf.math.reduce_mean(tf.clip_by_norm(grad, self.b), axis=0)
        clipped_grad = tf_l2_clip(grad, self.b)
        new_var_m = var - tf_gaussian_mech(clipped_grad, self.b/len(x_train), epsilon_i, delta_i) * lr_t
        #new_var_m = var - grad * lr_t
        
        new_var = new_var_m
        var.assign(new_var)

    
class RenyiDPGradientDescent(DPOptimizer):
    def __init__(self, epochs, alpha, epsilon_bar, b=3.0, learning_rate=0.01, name="RenyiDPGradientDescent", **kwargs):
        super().__init__(epochs,b=b, learning_rate=learning_rate, name=name, **kwargs)
        self._set_hyper("learning_rate", learning_rate)
        
        self.epsilon_bar = epsilon_bar
        self.alpha = alpha
        
    @tf.function
    def _resource_apply_dense(self, grad, var):
        var_dtype = var.dtype.base_dtype
        lr_t = self._decayed_lr(var_dtype)
        
        epsilon_bar_i = self.epsilon_bar / self.epochs
         
        clipped_grad = tf_l2_clip(grad, self.b)
        new_var_m = var - tf_gaussian_mech_RDP(clipped_grad, self.b/len(x_train), self.alpha, epsilon_bar_i) * lr_t
        
        new_var = new_var_m
        var.assign(new_var)

        
class ZeroConcentratedDPGradientDescent(DPOptimizer):
    def __init__(self, epochs, rho, b=3.0, learning_rate=0.01, name="ZeroConcentratedDPGradientDescent", **kwargs):
        super().__init__(epochs,b=b, learning_rate=learning_rate, name=name, **kwargs)
        self._set_hyper("learning_rate", learning_rate)
        
        self.rho = rho
        
    @tf.function
    def _resource_apply_dense(self, grad, var):
        var_dtype = var.dtype.base_dtype
        lr_t = self._decayed_lr(var_dtype)
        
        rho_i = self.rho / self.epochs
         
        clipped_grad = tf_l2_clip(grad, self.b)
        new_var_m = var - tf_gaussian_mech_zCDP(clipped_grad, self.b/len(x_train), rho_i) * lr_t
        
        new_var = new_var_m
        var.assign(new_var)

In [480]:
es = callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
ed_dp = EpsilonDeltaDPGradientDescent(1, 1000, 1e-5)
r_dp = RenyiDPGradientDescent(1, 500, 0.001)
zc_dp = ZeroConcentratedDPGradientDescent(5, 0.000001)
model.compile(optimizer=zc_dp, 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

model.fit(x=x_train,y=y_train, epochs=1, callbacks=[es], batch_size=64)


938/938 [==============================] - 14s 15ms/step - loss: 0.7842 - accuracy: 0.7753


In [481]:
t = tf.constant([1.0, 1.0, 1.0, 1.0])
tf_RDP_gaussian_mech(t, 0.0001, 500, 0.001)

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.9289808, 1.0607969, 0.9809418, 0.9453022], dtype=float32)>

In [482]:
t = tf.constant([1.0, 1.0, 1.0, 1.0])
tf_gaussian_mech_zCDP(t, 0.0001, 0.1)

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.99968153, 1.000329  , 1.0001874 , 1.0001769 ], dtype=float32)>